In [ ]:
import os
import json
import requests

In [ ]:
from sm_control import SMC

In [ ]:
## alo 로컬 실행
!python ../../main.py

In [ ]:
name="bolt_fastening_table_classification"

In [ ]:
URI = 'http://10.158.2.243:9999/'

In [ ]:
t = requests.get(URI + 'api/v1/workspaces')


In [ ]:
# ECR, S3 정보 획득
# api로 치환
s3_bucket = t.json()[0]['s3_bucket_name']
ecr_uri = t.json()[0]['ecr_base_path']
repo = name
ws_name = t.json()[0]['name']


In [ ]:
# 저장되는 솔메 패스를 입력
SM_path = "./solution_meta_raw.yaml"

In [ ]:
sm = SMC(s3_bucket, ecr_uri)

In [ ]:
sm.set_yaml()

In [ ]:
sm.set_sm_description(name, "testing", "s3://hahaha", "s3://hohoho", "params", "alo", "s3://icon")

In [ ]:
# s3 접근확인
sm.s3_access_check()

In [ ]:
pipeline = "train"

In [ ]:
# s3 데이터 업로드
# 이전에 있던 데이터는 지워집니다
sm.s3_upload(pipeline)

In [ ]:
sm.set_container_uri(pipeline) # uri도 그냥 입력되게 수정


In [ ]:
sm.set_cadidate_param(pipeline)

In [ ]:
sm.set_artifacts_uri(pipeline)
sm.set_resource(pipeline)

In [ ]:
pipeline = "inference"

In [ ]:
# s3 데이터 업로드
# 이전에 있던 데이터는 지워집니다
sm.s3_upload(pipeline)

In [ ]:
sm.set_container_uri(pipeline) # uri도 그냥 입력되게 수정


In [ ]:
sm.set_cadidate_param(pipeline)

In [ ]:
sm.set_artifacts_uri(pipeline)
sm.set_resource(pipeline)

In [ ]:
sm.set_wrangler()

In [ ]:
# 등록을 위한 형태 변경
data = {
  "metadata_json": sm.sm_yaml
}

In [ ]:
#json으로 변경
data =json.dumps(data)

In [ ]:
#등록
post_response = requests.post('http://10.158.2.243:9999/api/v1/rass', data=data)
post_response_json = post_response.json()
post_response_json